In [1]:
import _init_paths

import tensorflow as tf
import numpy as np
import nengo_dl
import nengo 

from configs.exp_configs import tf_exp_cfg as tf_cfg, nengo_dl_cfg as ndl_cfg
from utils.cnn_2d_utils import get_2d_cnn_model
from utils.nengo_dl_utils import get_nengo_dl_model
from utils.base_utils.exp_utils import get_grouped_slices_2d_pooling
from utils.base_utils.data_prep_utils import get_batches_of_exp_dataset, get_exp_dataset

In [2]:
model = get_2d_cnn_model((1, 28, 28), tf_cfg) # Channels first shape (1, 28, 28).
model[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 26, 26)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 13, 13)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 11, 11)        3480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 5, 5)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 3, 3)          6944      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [3]:
ndl_model = nengo_dl.Converter(model[0])

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2ab9d2560ef0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [5]:
print(dir(ndl_model))

['KerasTensorDict', 'TrackedDict', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_layer_converters', 'allow_fallback', 'converters', 'get_converter', 'inference_only', 'inputs', 'layers', 'max_to_avg_pool', 'model', 'net', 'outputs', 'register', 'scale_firing_rates', 'split_shared_weights', 'swap_activations', 'synapse', 'temporal_model', 'verify']


In [8]:
for lyr in ndl_model.model.layers:
  if lyr.name in ["conv2d", "conv2d_1"]:
    print(lyr.output.shape[1:])

(16, 26, 26)
(24, 11, 11)


In [20]:
nengo_dl.__version__

'3.4.0'

In [3]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=16, rows=26, cols=26)

In [4]:
len(grouped_slices) # 10816 == 16 x 26 x 26

In [5]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=24, rows=11, cols=11)

In [6]:
len(grouped_slices)# 24 x 11 x 11 = 2904 

# ###############################################################